<h6>LSTM cells with attention</h6>

<img src="lstm.png">
<img src="lstm1.png">
<img src="lstm2.png">


The basid LSTM equations are: 
    
    

In [13]:
#https://medium.com/datalogue/attention-in-keras-1892773a4f22

import numpy as np

import random
import json
import os

DATA_FOLDER = os.getcwd()

from faker import Faker
import babel
from babel.dates import format_date

FORMATS = ['short',
           'medium',
           'long',
           'full',
           'd MMM YYY',
           'd MMMM YYY',
           'dd MMM YYY',
           'd MMM, YYY',
           'd MMMM, YYY',
           'dd, MMM YYY',
           'd MM YY',
           'd MMMM YYY',
           'MMMM d YYY',
           'MMMM d, YYY',
           'dd.MM.YY',
           ]

LOCALES = babel.localedata.locale_identifiers()

def create_date():
    """
    """
    dt = Faker().date_object()
    
    try:
        human = format_date(dt, format=random.choice(FORMATS), locale=random.choice(LOCALES))
        case_change = random.randint(0,3)
        
        if case_change == 1:
            human = human.upper()
        elif case_change == 2:
            human = human.lower()
        
        machine = dt.isoformat()
    except AttributeError as e:
        return None,None,None
    return human, machine, dt

def create_dataset(dataset_name, n_examples, vocabulary=False):
    """
    """
    human_vocab = set()
    machine_vocab = set()
    
    with open(dataset_name, 'w') as f:
        for i in range(n_examples):
            h,m,_ = create_date()
            if h is not None:
                f.write('"' + h + '","' + m + '"\n')
                human_vocab.update(tuple(h))
                machine_vocab.update(tuple(h))
    
    if vocabulary:
        int2human = dict(enumerate(human_vocab))
        int2human.update = ({len(int2human):'<unk>', len(int2human)+1:'<eot>'})
        
        int2machine = dict(enumerate(machine_vocab))
        int2machine.update = ({len(int2machine):'<unk>', len(int2machine)+1:'<eot>'})
        
        human2int = {v:k for k, v in int2human.items()}
        machine2int = {v:k for k,v in int2machine.items()} 

        with open(os.path.join(DATA_FOLDER, 'human_vocab.json'), 'w') as f:
            json.dump(human2int, f)
        with open(os.path.join(DATA_FOLDER, 'machine_vocab.json'), 'w') as f:
            json.dump(machine2int, f)
        

In [17]:
create_dataset(os.path.join(DATA_FOLDER, 'training.csv'), 500000,
                   vocabulary=True)
create_dataset(os.path.join(DATA_FOLDER, 'validation.csv'), 1000)

AttributeError: 'dict' object attribute 'update' is read-only

In [ ]:
import numpy as np
import os
from keras.models import Model
from keras.layers import Dense, Embedding, Activation, Permute
from keras.layers import Input, Flatten, Dropout
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import TimeDistributed, Bidirectional
from .custom_recurrents import AttentionDecoder


def makeModel(pad_length=100,
              n_chars=105,
              n_labels=6,
              embedding_learnable=False,
              encoder_units=256,
              decoder_units=256,
              trainable=True,
              return_probabilities=False):
    
    input_ = Input(shape=(pad_length,), dtype='float32')
    input_embed = Embedding(n_chars, n_chars,
                            input_length=pad_length,
                            trainable=embedding_learnable,
                            weights=[np.eye(n_chars)],
                            name='OneHot')(input_)

    rnn_encoded = Bidirectional(LSTM(encoder_units, return_sequences=True),
                                name='bidirectional_1',
                                merge_mode='concat',
                                trainable=trainable)(input_embed)

    y_hat = AttentionDecoder(decoder_units,
                             name='attention_decoder_1',
                             output_dim=n_labels,
                             return_probabilities=return_probabilities,
                             trainable=trainable)(rnn_encoded)

    model = Model(inputs=input_, outputs=y_hat)

    return model
l